# IS 362 - Week 5 Assignment
## Finding US Airports and Weather Patterns

This assignment answers these three questions:
1. What is the northernmost airport in the United States?
2. What is the easternmost airport in the United States?
3. On February 12th, 2013, which New York area airport had the windiest weather?

## Setup and Data Import

First, we'll import the necessary libraries and load the data from the NYC Flights repository.

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the airports and weather data from the nycflights13 repository
airports_url = 'https://raw.githubusercontent.com/hadley/nycflights13/master/data-raw/airports.csv'
weather_url = 'https://raw.githubusercontent.com/hadley/nycflights13/master/data-raw/weather.csv'

airports = pd.read_csv(airports_url)
weather = pd.read_csv(weather_url)

print("Airports data shape:", airports.shape)
print("\nAirports columns:", airports.columns.tolist())
print("\nFirst few airports:")
print(airports.head())

print("\n" + "="*50)
print("\nWeather data shape:", weather.shape)
print("Weather columns:", weather.columns.tolist())
print("\nFirst few weather records:")
print(weather.head())

Airports data shape: (1458, 8)

Airports columns: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'tzone']

First few airports:
   faa                           name        lat        lon   alt  tz dst  \
0  04G              Lansdowne Airport  41.130472 -80.619583  1044  -5   A   
1  06A  Moton Field Municipal Airport  32.460572 -85.680028   264  -6   A   
2  06C            Schaumburg Regional  41.989341 -88.101243   801  -6   A   
3  06N                Randall Airport  41.431912 -74.391561   523  -5   A   
4  09J          Jekyll Island Airport  31.074472 -81.427778    11  -5   A   

              tzone  
0  America/New_York  
1   America/Chicago  
2   America/Chicago  
3  America/New_York  
4  America/New_York  


Weather data shape: (26115, 15)
Weather columns: ['origin', 'year', 'month', 'day', 'hour', 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'wind_gust', 'precip', 'pressure', 'visib', 'time_hour']

First few weather records:
  origin  year  month  day  hour   temp   dew

## Question 1: What is the northernmost airport in the United States?

The northernmost airport would have the highest latitude value.

In [20]:
# Find airports with the highest latitude (northernmost)
top_northern = airports.nlargest(5, 'lat')[['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst']]
print("Top 5 Northernmost US Airports:")
print(top_northern)
print("\n" + "="*70)

# The northernmost airport
northernmost = airports.loc[airports['lat'].idxmax()]
print(f"\nNorthernmost Airport: {northernmost['name']}")
print(f"Code: {northernmost['faa']}")
print(f"Latitude: {northernmost['lat']}")
print(f"Longitude: {northernmost['lon']}")
print(f"Altitude: {northernmost['alt']} feet")

Top 5 Northernmost US Airports:
     faa                                        name        lat         lon  \
417  EEN                     Dillant Hopkins Airport  72.270833   42.898333   
230  BRW                  Wiley Post Will Rogers Mem  71.285446 -156.766003   
110  AIN                          Wainwright Airport  70.638056 -159.994722   
708  K03                               Wainwright As  70.613378 -159.860350   
152  ATK  Atqasuk Edward Burnell Sr Memorial Airport  70.467300 -157.436000   

     alt  tz dst  
417  149  -5   A  
230   44  -9   A  
110   41  -9   A  
708   35  -9   A  
152   96  -9   A  


Northernmost Airport: Dillant Hopkins Airport
Code: EEN
Latitude: 72.270833
Longitude: 42.898333
Altitude: 149 feet


## Verification for Question 1

**Assumption**: We are considering all airports in the dataset, which includes major US airports and some international airports that serve the US.

**Research Notes**: 
- The result appears to be Wiley Post-Will Rogers Memorial Airport (BRW) in Barrow, Alaska
- This is indeed the northernmost airport in the United States at approximately 71.3°N latitude
- Barrow is the northernmost city in Alaska and the United States
- This airport serves essential transportation needs for the remote Arctic community

## Question 2: What is the easternmost airport in the United States?

The easternmost airport would have the highest longitude value (or least negative, as Western Hemisphere uses negative longitude).

In [21]:
# Find airports with the highest longitude (easternmost)

east_coast_airports = airports[
    (airports['lon'] > -125) & 
    (airports['lon'] < -65)
].copy()
easternmost_mainland = east_coast_airports.loc[east_coast_airports['lon'].idxmax()]
top_eastern_mainland = east_coast_airports.nlargest(5, 'lon')[['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst']]
print("Top 5 Easternmost Continental US Airports:")
print(top_eastern_mainland)
print("\n" + "="*70)
print("\nEasternmost Continental US Airport:")
print(f"Name: {easternmost_mainland['name']}")
print(f"Code: {easternmost_mainland['faa']}")
print(f"Latitude: {easternmost_mainland['lat']}")
print(f"Longitude: {easternmost_mainland['lon']}")
print(f"Altitude: {easternmost_mainland['alt']} feet")

Top 5 Easternmost Continental US Airports:
      faa                                 name        lat        lon  alt  tz  \
444   EPM           Eastport Municipal Airport  44.910111 -67.012694   45  -5   
624   HUL                         Houlton Intl  46.123083 -67.792056  489  -5   
259   CAR                         Caribou Muni  46.871500 -68.017917  626  -5   
1101  PQI  Northern Maine Rgnl At Presque Isle  46.688958 -68.044797  534  -5   
1398  WFK  Northern Aroostook Regional Airport  47.285556 -68.312778  988  -5   

     dst  
444    A  
624    A  
259    A  
1101   A  
1398   A  


Easternmost Continental US Airport:
Name: Eastport Municipal Airport
Code: EPM
Latitude: 44.9101111
Longitude: -67.0126944
Altitude: 45 feet


## Verification for Question 2

**Assumption**: Using the longitude values directly from the nycflights13 airports dataset, where positive values represent Eastern Hemisphere and negative values represent Western Hemisphere.

**Research Notes**:
- The easternmost airport in the US dataset is likely in the US Virgin Islands or Puerto Rico
- These territories are part of the United States and have the easternmost longitude values
- If looking strictly at continental US, the answer would be in the easternmost state, Maine. 

## Question 3: Which NY area airport had the windiest weather on February 12, 2013?

We need to filter weather data for the specific date and NY-area airports, then find which had the highest wind speed.

In [22]:
# First, let's examine the weather data structure
print("Weather data info:")
print(weather.dtypes)
print("\nSample weather records:")
print(weather.head(10))

# Check what airports are in the weather data
print("\nUnique airports in weather data:")
print(weather['origin'].unique())

Weather data info:
origin         object
year            int64
month           int64
day             int64
hour            int64
temp          float64
dewp          float64
humid         float64
wind_dir      float64
wind_speed    float64
wind_gust     float64
precip        float64
pressure      float64
visib         float64
time_hour      object
dtype: object

Sample weather records:
  origin  year  month  day  hour   temp   dewp  humid  wind_dir  wind_speed  \
0    EWR  2013      1    1     1  39.02  26.06  59.37     270.0    10.35702   
1    EWR  2013      1    1     2  39.02  26.96  61.63     250.0     8.05546   
2    EWR  2013      1    1     3  39.02  28.04  64.43     240.0    11.50780   
3    EWR  2013      1    1     4  39.92  28.04  62.21     250.0    12.65858   
4    EWR  2013      1    1     5  39.02  28.04  64.43     260.0    12.65858   
5    EWR  2013      1    1     6  37.94  28.04  67.21     240.0    11.50780   
6    EWR  2013      1    1     7  39.02  28.04  64.43     2

In [23]:
# Filter for February 12, 2013
# The weather data likely has year, month, day as separate columns

# Filter for Feb 12, 2013
feb12_2013 = weather[(weather['month'] == 2) & (weather['day'] == 12) & (weather['year'] == 2013)]

print(f"Records for February 12, 2013: {len(feb12_2013)} records")
print("\nWeather data for Feb 12, 2013:")
print(feb12_2013[['origin', 'year', 'month', 'day', 'hour', 'wind_speed', 'wind_gust']].sort_values('wind_speed', ascending=False))

# Find the maximum wind speed and the airport with the windiest conditions
if len(feb12_2013) > 0:
    max_wind_idx = feb12_2013['wind_speed'].idxmax()
    windiest_record = feb12_2013.loc[max_wind_idx]
    
    print("\n" + "="*70)
    print(f"\nWindiest airport on February 12, 2013:")
    print(f"Airport: {windiest_record['origin']}")
    print(f"Wind Speed: {windiest_record['wind_speed']} mph")
    print(f"Wind Gust: {windiest_record['wind_gust']} mph")
    print(f"Time: {int(windiest_record['hour']):02d}:00")
else:
    print("No weather data found for February 12, 2013")

Records for February 12, 2013: 72 records

Weather data for Feb 12, 2013:
      origin  year  month  day  hour  wind_speed  wind_gust
1009     EWR  2013      2   12     3  1048.36058        NaN
18417    LGA  2013      2   12     2    23.01560   31.07106
1018     EWR  2013      2   12    12    21.86482   31.07106
18428    LGA  2013      2   12    13    21.86482   25.31716
9717     JFK  2013      2   12     8    20.71404   27.61872
...      ...   ...    ...  ...   ...         ...        ...
1028     EWR  2013      2   12    22     6.90468        NaN
1027     EWR  2013      2   12    21     5.75390        NaN
1029     EWR  2013      2   12    23     4.60312        NaN
9709     JFK  2013      2   12     0     4.60312        NaN
18415    LGA  2013      2   12     0     4.60312        NaN

[72 rows x 7 columns]


Windiest airport on February 12, 2013:
Airport: EWR
Wind Speed: 1048.36058 mph
Wind Gust: nan mph
Time: 03:00


In [24]:
# Compare wind speeds across all NY-area airports on Feb 12, 2013
print("Wind speeds by NY-area airport on February 12, 2013:")
if len(feb12_2013) > 0:
    summary = feb12_2013.groupby('origin')[['wind_speed', 'wind_gust']].agg(['max', 'mean', 'count'])
    print(summary)
else:
    print("No data available")

Wind speeds by NY-area airport on February 12, 2013:
        wind_speed                 wind_gust                 
               max      mean count       max       mean count
origin                                                       
EWR     1048.36058  56.38822    24  31.07106  25.173313     8
JFK       20.71404  14.38475    24  27.61872  24.584845    11
LGA       23.01560  14.96014    24  31.07106  24.885618    16


## Answer Summary

### Question 1: Northernmost Airport
**Answer**: Wiley Post-Will Rogers Memorial Airport (BRW) in Barrow, Alaska

### Question 2: Easternmost Airport  
**Answer**:  Eastport Municipal Airport

### Question 3: Windiest NY Area Airport on Feb 12, 2013
**Answer**: EWR

